In [1]:
from openai import OpenAI
import json, os, sys
import os
import re
import random

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def generate_questions(chunk, client, model="gpt-4o"):
    prompt = f"""
你是一位新闻分析专家，擅长创建有助于区分文章主题的多项选择题答案。
我有一些文章摘要，我需要基于这些摘要生成独特的、全面的多项选择题。

你的任务是生成问题和答案，要求如下：

	•	有助于区分不同文章的主题。
	•	问题适用于所有文章，而不是针对某一篇文章。
	•	每批问题都是独特的，不与其他批重复。
	•	每个问题都包含“以上皆非”作为选项之一。

以下是我需要的格式和示例：

1. 做了什么？
   - A. 中国海警扣押了菲律宾军用飞机投下的四个食品包之一，这些食品包是为菲律宾海军人员在领土哨所准备的。在发现包裹里装的是食物后，他们将其扔进了海里。
   - B. 据报道，菲律宾士兵在向搁浅的“马德雷山”号船进行补给任务时，曾用枪指向中国海警人员。中国海警在这次补给行动中作出反应，包括观察到菲律宾士兵在船甲板上持枪。
   - C. 土耳其外长哈坎·菲丹开始了对中国的访问，并表示优先支持哈马斯对抗以色列并增加与中国的贸易，但未谴责维吾尔种族灭绝。
   - D. 菲律宾正与美国和日本合作，以确保在与中国海上力量的紧张局势中保持西菲律宾海（WPS）的自由和安全。
   - E. 以上皆非

摘要： {chunk}

生成尽可能多的独特问题基于上述摘要。
"""
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [4]:
def process_batches(input_file, output_file, client, model):
    # Read the article summaries from the input JSON file
    with open(input_file, "r") as f:
        articles = json.load(f)

    # Shuffle the data
    random.shuffle(articles)

    # Extract summaries from the articles
    article_summaries = [article["summary"] for article in articles]
    num_summaries = len(article_summaries)
    batch_size = 200

    responses = []

    # Process the article summaries in batches of 100
    for i in range(0, num_summaries, batch_size):
        batch = article_summaries[i : i + batch_size]
        chunk = json.dumps(batch)  # Convert the batch to a JSON string

        questions = generate_questions(chunk, client, model=model)
        responses.append(
            {
                "batch_start": i,
                "batch_end": min(i + batch_size - 1, num_summaries - 1),
                "questions": questions,
            }
        )

    # Write the generated questions to the output JSON file
    with open(output_file, "w") as f:
        json.dump(responses, f, ensure_ascii=False, indent=4)

In [5]:
input_file = "combined_Russo_Ukrainian_War.json"
output_file = "output_questions_zho.json"

# Assuming you have your OpenAI API client initialized as `client`
process_batches(input_file, output_file, client, model="gpt-4o")

In [6]:
def extract_questions(batch_questions):
    questions = {}
    q_count = 1
    for batch in batch_questions:
        batch_qs = batch["questions"].split("\n\n")
        for q in batch_qs:
            match = re.match(
                r"\d+\. (.+?)\n\s+- A\. (.+?)\n\s+- B\. (.+?)\n\s+- C\. (.+?)\n\s+- D\. (.+?)\n\s+- E\. (.+?)$",
                q.strip(),
                re.DOTALL,
            )
            if match:
                question = match.group(1).strip()
                choices = {
                    "A": match.group(2).strip(),
                    "B": match.group(3).strip(),
                    "C": match.group(4).strip(),
                    "D": match.group(5).strip(),
                    "E": match.group(6).strip(),
                }
                if question not in questions:
                    questions[question] = choices
    return questions


def convert_to_json2_format(unique_questions):
    formatted_questions = {}
    q_num = 1
    for question, choices in unique_questions.items():
        formatted_questions[f"Q{q_num}"] = {"question": question, "choices": choices}
        q_num += 1
    return formatted_questions


def process_json(input_json):
    unique_questions = extract_questions(input_json)
    return convert_to_json2_format(unique_questions)

In [7]:
# Load JSON 1
with open("output_questions_zho.json", "r") as f:
    json1 = json.load(f)

# Process JSON 1 to get JSON 2 format
json2_format = process_json(json1)

# Save the result to a new JSON file
with open("formated_output_questions_zho.json", "w") as f:
    json.dump(json2_format, f, ensure_ascii=False, indent=4)

In [8]:
print(json1)

[{'batch_start': 0, 'batch_end': 199, 'questions': '1. 文章主要讨论了什么？\n   - A. 北约为应对特朗普可能重返白宫而制定的“B计划”，包括要求欧洲成员国增加国防开支和建立新的军事指挥中心。\n   - B. 英国首相里希·苏纳克呼吁选民防止工党在即将到来的大选中获得“超级多数”，并强调投票给保守党以避免更高的税收。\n   - C. 俄罗斯总统普京在越南的新闻发布会上表示，俄罗斯不会从乌克兰撤军，并强调只有乌克兰完全失败才能结束战争。\n   - D. 明阿平谷的居民要求恢复和加快当地发展项目，并警告如果他们的要求得不到满足，将进行抗议。\n   - E. 以上皆非\n\n2. 文章提到的主要人物是谁？\n   - A. 北约秘书长延斯·斯托尔滕贝格\n   - B. 英国首相里希·苏纳克\n   - C. 俄罗斯总统弗拉基米尔·普京\n   - D. 土耳其外长哈坎·菲丹\n   - E. 以上皆非\n\n3. 文章中提到的主要事件是什么？\n   - A. 北约计划在德国威斯巴登建立新的军事指挥中心以支持乌克兰。\n   - B. 英国首相里希·苏纳克呼吁选民防止工党在即将到来的大选中获得“超级多数”。\n   - C. 俄罗斯总统普京在越南的新闻发布会上表示，俄罗斯不会从乌克兰撤军。\n   - D. 明阿平谷的居民要求恢复和加快当地发展项目。\n   - E. 以上皆非\n\n4. 文章中提到的主要国家或地区是哪里？\n   - A. 乌克兰\n   - B. 英国\n   - C. 俄罗斯\n   - D. 明阿平谷\n   - E. 以上皆非\n\n5. 文章中提到的主要组织是什么？\n   - A. 北约\n   - B. 英国保守党\n   - C. 俄罗斯政府\n   - D. 明阿平谷居民委员会\n   - E. 以上皆非\n\n6. 文章中提到的主要问题是什么？\n   - A. 北约为应对特朗普可能重返白宫而制定的“B计划”\n   - B. 英国首相里希·苏纳克呼吁选民防止工党在即将到来的大选中获得“超级多数”\n   - C. 俄罗斯总统普京在越南的新闻发布会上表示，俄罗斯不会从乌克兰撤军\n   - D. 明阿平谷的居民要求恢复和加快当地发展项目\n   - E. 以上皆非